In [2]:
from rocketcea.cea_obj_w_units import CEA_Obj
from proptools import nozzle  
import numpy as np

In [56]:
class Engine:
    def __init__(self,
                 fuelName:str,
                 oxName:str,
                 MR:float,
                 Pc:float,
                 eps:float,
                 nozzleDiam = None,
                 At=None):
        """
        fuel_name: 
        ox_name (oxidizer name):
        mr (mixture ratio):
        nozzle_diam (nozzle throat diameter):
        esp (nozzle expansion ratio)
        At (Area - throat)
        """
        
        self.fuel_name = fuelName
        self.ox_name = oxName
        self.MR = MR
        self.Pc = Pc
        self.nozzleDiam = nozzleDiam
        self.eps = eps
        self.At = At

        # Podemos escolher informar o diametro do throat ou a area
        if ((self.At == None) and (self.nozzleDiam != None)):
            self.At = np.pi * ((self.nozzleDiam/2) ** 2)
        elif ((self.At == None) and (self.nozzleDiam == None)):
            raise("Informar At ou nozzleDiam")
        elif ((self.At != None) and (self.nozzleDiam != None)):
            raise("Informar apenas 1: At ou nozzleDiam")

        # Pressao precisa estar em MPa apesar e dados precisam ser fornecidos na ordem de 1e6
        self.ceaObj = CEA_Obj( oxName=oxName, fuelName=fuelName, pressure_units='MPa', cstar_units='m/s', temperature_units='K')

    def calcGasProperties(self):
        # mw -> Molecular weight
        # Cstar -> Characteristc velocity (Isp * g0 / Cf)
        IspVac, Cstar, Tc, mw, gamma = self.ceaObj.get_IvacCstrTc_ChmMwGam(Pc=self.Pc, MR=self.MR, eps=self.eps)
        return
    
    def calcEngineProperties(self):
        IspVac, Cstar, Tc, mw, gamma = self.ceaObj.get_IvacCstrTc_ChmMwGam(Pc=self.Pc, MR=self.MR, eps=self.eps)
        m_molar = mw/1000
        IspSea = self.ceaObj.estimate_Ambient_Isp(Pc=self.Pc, MR=self.MR, eps=self.eps, Pamb=1e5)
        Pc = self.Pc 


        Pe = Pc * nozzle.pressure_from_er(self.eps, gamma)
        # Empuxo no vacuo (N)
        ThrustVac = nozzle.thrust(A_t = self.At,
                                  p_c = Pc,
                                  p_e = Pe,
                                  p_a = 0,
                                  gamma=gamma,
                                  er = self.eps)
        # Empuxo no nivel do mar (N)
        ThrustSea = nozzle.thrust(A_t = self.At,
                                  p_c = Pc,
                                  p_e = Pe,
                                  p_a = 1e5,
                                  gamma=gamma,
                                  er = self.eps)
        # Fluxo de mass (kg/s)
        mass_flow = nozzle.mass_flow(A_t = self.At,
                                     p_c = Pc,
                                     T_c = Tc,
                                     gamma = gamma,
                                     m_molar = m_molar
                                     )   

        print("Isp Vac: " + str(IspVac))
        print("Isp Sea: " + str(IspSea))
        print("Mass flow: " + str(mass_flow))
        print("Thrust Vac: " + str(ThrustVac/1000))
        print("Thrust Sea: " + str(ThrustSea/1000))

engine = Engine(fuelName="RP-1", 
                oxName="LOX",
                MR=2.36,
                Pc=9.7*1e6 ,
                At=0.042,
                eps=21.4)

engine.calcEngineProperties()


    


Isp Vac: 305.5677676625125
Isp Sea: (274.39544109463185, 'OverExpanded (Pe=1.39683e+07)')
Mass flow: 294.4607412091103
Thrust Vac: 905.7796996285897
Thrust Sea: 815.8996996285897


In [54]:
engine = Engine(fuelName="RP-1", 
                oxName="LOX",
                MR=2.17,
                Pc=6.14*1e6 ,
                At=0.042,
                eps=117)

engine.calcEngineProperties()


Isp Vac: 343.62656740037494
Isp Sea: (138.72158902076214, 'Separated (Psep=4.24381e+06, epsSep=64.2578)')
Mass flow: 186.7508523473767
Thrust Vac: 714.5433289320969
Thrust Sea: 223.14332893209672
